In [ ]:
!pip install natasha
!pip install Wikipedia-API
!pip install wikipedia


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 46.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=7e9a99328e619ac4cc565817ac2f6671f58aeeb292b3125172933c3b2fa0f9ac
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=312eca34a68d01198970715baad8e07717a14fd232c5c8994ab173d5b4de4a40
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built docopt intervaltree
  Preparing metadata (set

In [ ]:
from natasha import (Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, PER, NamesExtractor, DatesExtractor, MoneyExtractor, AddrExtractor, Doc)
import wikipediaapi
import wikipedia
import re

In [ ]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
data_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
address_extractor = AddrExtractor(morph_vocab)

In [ ]:
#ищет локацию, организацию или имя
def exctract_NER(title):
  doc=Doc(title)
  #разбивка текста на предложения
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  doc.tag_ner(ner_tagger)

  #for word in doc.tokens:
  #  word.lemmatize(morph_vocab)


  return [[x.text, x.type] for x in doc.spans]

In [ ]:
def find(link_dict,name,val):
  find_dict={}
  for id in link_dict.keys():
    if link_dict[id][name]==val:
      find_dict[id] = link_dict[id]
  return find_dict

def find_links(link_dict):
  roots=find(link_dict,'rel','root')
  id_first_root=list(roots.keys())[0]
  #поиск связанных с рутом слов
  predicate=find(link_dict,'head_id',id_first_root)
  #поиск сказуемого, связанного с рутом
  link_predicate=find(predicate,'rel','nsubj')
  if len(list(link_predicate.keys()))==0: return {}
  id_first_link_predicate=list(link_predicate.keys())[0]
  return f"{roots[id_first_root]['text']} {link_predicate[id_first_link_predicate]['text']}"

def fill_link_dict(sent):
    link_dict={}
    for _ in sent.syntax:
      for word in _:
        link_dict[word.id]={"text":word.text,"head_id":word.head_id,"rel":word.rel}
    return link_dict


def analise_links(sentances):
  links=[]
  for sent in sentances:
    link_dict=fill_link_dict(sent)
    main_link=find_links(link_dict)
    links.append(main_link)
  return links


In [ ]:
#ищет подлежащее и сказуемое
def find_word_links(discription):
  doc=Doc(discription)
  #разбивка текста на предложения
  doc.segment(segmenter)
  #разбивка предложений на части речи
  doc.tag_morph(morph_tagger)
  #выявления связей между словами
  doc.parse_syntax(syntax_parser)
  links=analise_links(doc.sents)
  return links

In [ ]:
def unification_X_words(all_extracts):
  pos_all_extracts=[x.pos for x in all_extracts]
  idx_X_to_unif=[]
  unif_X_words=[]
  for i in range(len(pos_all_extracts)-1):
    pos=pos_all_extracts[i]
    next_pos=pos_all_extracts[i+1]
    if pos==next_pos=='X':
      idx_X_to_unif.append(i)
      idx_X_to_unif.append(i+1)
    elif (pos=="X"):
      idx_X_to_unif.append(i)
    if ('X'==pos) and (next_pos!='X'):
      unif_X_words.append(' '.join([all_extracts[idx_X].text for idx_X in set(idx_X_to_unif)]))
      idx_X_to_unif=[]
  return unif_X_words

In [ ]:
#выделяет части речи
def exctract_partOfSpeech(text, pos=[], sort=True):
  doc=Doc(text)
  #разбивка текста на предложения
  doc.segment(segmenter)
  #разбивка предложений на части речи
  doc.tag_morph(morph_tagger)

  all_extracts=[]
  sort_extracts=[]
  for token in doc.tokens:
    token.lemmatize(morph_vocab)
    if (token.pos in pos):
       if not sort: print(token.lemma, token.pos)
       sort_extracts.append(token.lemma)

    all_extracts.append(token)
  if not sort:
    return all_extracts
  if 'X' in pos:
    #return sort_extracts
    return unification_X_words(all_extracts)

  return sort_extracts

In [ ]:
def request_wiki(name,language):
  wiki_wiki = wikipediaapi.Wikipedia('reqests (reqest@example.com)',language)
  #если ни один X_word не был объединен с другим
  if len(name)<2:
    return 0

  #Если название чего-либо было случайно обрезано, то на том шаге исчитаться к какому названию в ввикипедии ближе всего и замаеняется
  page = wiki_wiki.page(name)

  #Проверка существует ли страница в вики
  if not page.exists():
    return 0
  return page

def prepare_wiki_requst(name):
  try:
    name=wikipedia.search(name)[0]
  except:
    pass
  page_en=request_wiki(name,'en')
  page_ru=request_wiki(name,'ru')


  analise_dict={}
  if (page_en==0) or (page_ru==0): return analise_dict

  analise_dict['category']=list(page_en.categories.keys())[0]#[:9]
  #analise_dict['summary']=page_ru.summary
  analise_dict['NER']=exctract_NER(page_ru.summary)

  return analise_dict

In [ ]:
def clear_text(text):
 cleaned_text = re.sub(r'[^-а-яА-Яa-zA-Z.,/ ]', '', re.sub(r'\([^)]*\)|\{[^}]*\}|\[[^\]]*\]', '', text))
 return cleaned_text

In [ ]:
def full_analise(title,discription):
  analise_dict={}
  title=clear_text(title)
  #выделяються собственные существительные
  analise_dict['names']=exctract_partOfSpeech(title, ['PROPN'])

  #выделение всех существительных
  analise_dict['NOUN']=exctract_partOfSpeech(title, ['NOUN'])

  #выделение NER
  analise_dict['NER']=exctract_NER(title)

  #поиск слов в вики, которые не удалось классифицировать как части речи
  wiki={}
  dont_find_words=exctract_partOfSpeech(title, ['X'])
  for dfw in dont_find_words:
    wiki[dfw]=prepare_wiki_requst(dfw)

  analise_dict['wiki']=wiki

  return analise_dict



In [ ]:
title="""Присутствует спойлеры и ненормативная лексика. В этой нарезке вы увидите: Как использовать паучье чутье в Spider-Man, косплей на человека-паука, мнение об игре и другие забавные моменты"""
#title="Прохождение Red Dead Redemption 2 | RDR2 | TheHedwigGames"
discription="Прохождение Red Dead Redemption 2 | RDR2 | TheHedwigGames"

In [ ]:
full_analise(title,discription)

<generator object exctract_NER.<locals>.<genexpr> at 0x7d2c3a50eea0>
<generator object exctract_NER.<locals>.<genexpr> at 0x7d2c3a50e880>


{'names': [],
 'NOUN': ['спойлер',
  'лексика',
  'нарезка',
  'чутье',
  'человек-паук',
  'мнение',
  'игра',
  'момент'],
 'NER': [['Spider-Man', 'ORG']],
 'wiki': {'Spider-Man': {'category': 'Category:1962 comics debuts',
   'NER': [['Пи́тер Бе́нджамин Па́ркер', 'PER'],
    ['Peter Benjamin Parker)', 'PER'],
    ['Marvel Comics', 'ORG'],
    ['Стэном Ли', 'PER'],
    ['Стивом Дитко', 'PER'],
    ['Ли', 'PER'],
    ['Дитко', 'PER'],
    ['Стив Роджерс', 'PER'],
    ['Робина', 'PER'],
    ['Бена', 'PER'],
    ['Marvel', 'ORG'],
    ['Питер Паркер', 'PER'],
    ['Питера Паркера', 'PER'],
    ['Человек-паук', 'PER'],
    ['Тоби Магуайр', 'PER'],
    ['Сэма Рэйми', 'PER'],
    ['Эндрю Гарфилд', 'PER'],
    ['Марка Уэбба', 'PER'],
    ['Том Холланд', 'PER'],
    ['Marvel', 'ORG'],
    ['Рив Карни', 'PER'],
    ['Питера Паркера', 'PER']]}}}